# Example 2 - Pine Island Glacier

This notebook provides an example of a typical ISSM workflow to complete the following:

1. Build and parameterise an ISSM model
2. Conduct a basal friction inversion
3. Run a transient stress balance simulation

In [1]:
import os
import pyissm
import datapool as dp
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

## Setup your modelling environment

Some High-Performance Computing (HPC) infrastructure places limitations on user read/write permissions. To make this notebook flexible for all users, please define some variables used internally in this notebook. The following variables must be defined in the cell below:

- `tutorial_dir = <PATH_TO_NOTEBOOK>` where this notebook is located. By default, this is assumed to be `~/pyISSM/tutorials`
- `tutorial_asset_dir = <PATH_TO_ASSETS>` where all tutorials assets are located. By default, this is assumed to be `~/pyISSM/tutorials/assets`
- `execution_dir = <PATH_TO_DIRECTORY>` where model files will be saved. You must have `rwx` permissions for this directory. By default, this is assumed to be `~/pyISSM/tutorials/models`

NOTE: `execution_dir` must be different from the current working directory of your Python kernel.

In [3]:
tutorial_dir = str(Path.home() / 'pyISSM' / 'tutorials')
asset_dir = tutorial_dir + '/assets'
execution_dir = tutorial_dir + '/models'

# Check that execution directory exists. If not, create it
if not os.path.isdir(execution_dir):
    os.mkdir(execution_dir)

---
## 1. Model mesh

In this example, we first generate a uniform mesh with a resolution of 10 km. We then refine this mesh based on the observed velocity field using anisotropic mesh refinement. At each step, we plot the mesh and velocity field.

In [ ]:
# Define mesh parameters
hinit = 10000
hmax = 40000
hmin = 5000
gradation = 1.7
err = 8

# Generate initial uniform mesh
md = pyissm.model.mesh.bamg(pyissm.model.Model(),
                            domain = asset_dir + '/Exp/PIG_DomainOutline.exp',
                            hmax = hinit)

In [ ]:
# Plot initial uniform mesh
fig, ax = pyissm.plot.plot_mesh2d(md)
ax.set_title('Initial uniform model mesh')

Here, we load the velocity dataset and interpolate this onto the model mesh. Here we use the ACCESS Cryosphere Data Pool to access the data on NCI Gadi. If running this notebook locally, simply load the dataset using `xarray`.

In [ ]:
# Load velocity
catalog = dp.catalog.DataCatalog()
velocity = catalog.load_dataset('measures_insar_based_antarctica_ice_velocity_map', version = 'v2')

# Assign velocity to model
vx_mesh = pyissm.data.interp.xr_to_mesh(velocity, 'VX', md.mesh.x, md.mesh.y, fill_nan = True)
vy_mesh = pyissm.data.interp.xr_to_mesh(velocity, 'VY', md.mesh.x, md.mesh.y, fill_nan = True)
vel_mesh = np.sqrt(vx_mesh**2 + vy_mesh**2)

In [ ]:
# Visualise velocity
fig, ax = pyissm.plot.plot_model_field(md,
                                       vel_mesh,
                                       show_mesh = True,
                                       mesh_kwargs = {'color': 'white'},
                                       show_cbar = True,
                                       cbar_kwargs = {'label': 'Ice surface velocity (m yr$^{-1}$)'})
ax.set_title('Observed velocity')

In [ ]:
## Adapt the mesh based on the velocity
md = pyissm.model.mesh.bamg(md,
                            hmax = hmax,
                            hmin = hmin,
                            gradation = gradation,
                            field = vel_mesh,
                            err = err)

In [ ]:
fig, ax = pyissm.plot.plot_mesh2d(md)
ax.set_title('Refined model mesh')

In [ ]:
## Save the model
pyissm.model.io.save_model(md, tutorial_dir + '/ex2_PIG_mesh.nc')

# 2. Model Mask

In this example, we use the `mask` variable from BedMachine v3 to define regions of floating and grounded ice. 

In [ ]:
## Load the model
md = pyissm.model.io.load_model(tutorial_dir + '/ex2_PIG_mesh.nc')

In [ ]:
## Load bedmachine v3
bm = catalog.load_dataset('measures_bedmachine_antarctica')

# Interpolate mask to model mesh
## NOTE: Use interpolation_type = 'nearest' to retain unique values for mask
mask = pyissm.data.interp.xr_to_mesh(bm, 'mask', md.mesh.x, md.mesh.y, interpolation_type = 'nearest')
fig, ax = pyissm.plot.plot_model_field(md, mask, show_cbar = True)
ax.set_title('Native BedMachine v3 mask')

In [ ]:
# Assign mask to model levelsets
ice_levelset = np.full(md.mesh.numberofvertices, np.nan)
ocean_levelset = np.full(md.mesh.numberofvertices, np.nan)

## Define ice / no-ice areas
ice_levelset[mask > 0] = -1
ice_levelset[np.isnan(ice_levelset)] = 1

## Define ocean / no-ocean areas
ocean_levelset[mask == 2] = 1
ocean_levelset[mask == 3] = -1
ocean_levelset[mask == 0] = -1

## Assign to model
md.mask.ice_levelset = ice_levelset
md.mask.ocean_levelset = ocean_levelset

# Visualise elements
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (14, 4), sharey = True)
ax1 = pyissm.plot.plot_model_elements(md, md.mask.ice_levelset, md.mask.ocean_levelset, type = 'ice_elements', ax = ax1)
ax2 = pyissm.plot.plot_model_elements(md, md.mask.ice_levelset, md.mask.ocean_levelset, type = 'floating_ice_elements', ax = ax2, ylabel = '')
ax3 = pyissm.plot.plot_model_elements(md, md.mask.ice_levelset, md.mask.ocean_levelset, type = 'grounding_line_elements', ax = ax3, ylabel = '')

In [ ]:
## Save the model
pyissm.model.io.save_model(md, tutorial_dir + '/ex2_PIG_mask.nc')

## 3. Parameterize Model

In [ ]:
## Load the model
md = pyissm.model.io.load_model(tutorial_dir + '/ex2_PIG_mask.nc')

In [ ]:
## Parameterize the model
md = pyissm.model.param.parameterize(md, asset_dir + '/Param/pig_param.py')

In [ ]:
## Set flow equation (SSA)
md = pyissm.model.param.set_flow_equation(md, SSA = 'all')

In [ ]:
## Visualise various parameterised fields
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (12, 10), sharex = True, sharey = True)
ax1 = pyissm.plot.plot_model_bc(md, ax = ax1, xlabel = '', legend_kwargs = {'fontsize': 7, 'title_fontsize': 8, 'loc': 'lower right'})
ax2 = pyissm.plot.plot_model_field(md, md.geometry.bed, ax = ax2, show_cbar = True, xlabel='', ylabel = '', cbar_kwargs = {'label': 'Bed elevation (m)'})
ax3 = pyissm.plot.plot_model_field(md, md.geometry.thickness, ax = ax3, show_cbar = True, cbar_kwargs = {'label': 'Ice thickness (m)'})
ax4 = pyissm.plot.plot_model_field(md, md.smb.mass_balance, ax = ax4, show_cbar = True, ylabel='', cbar_kwargs = {'label': 'Surface mass balance (m[ie])'})

ax1.set_title('Stress balance boundary conditions')
ax2.set_title('md.geometry.bed')
ax3.set_title('md.geometry.thickness')
ax4.set_title('md.smb.mass_balance')

In [ ]:
## Save the model
pyissm.model.io.save_model(md, tutorial_dir + '/ex2_PIG_param.nc')

# 4. Perform friction inversion

In [4]:
## Load model
md = pyissm.model.io.load_model(tutorial_dir + '/ex2_PIG_param.nc')

No classtype found for solidearth.settings.
No classtype found for solidearth.lovenumbers.
No classtype found for solidearth.rotational.
No classtype found for toolkits.DefaultAnalysis.
No classtype found for toolkits.RecoveryAnalysis.
No classtype found for qmu.method.
No classtype found for qmu.statistics.
No classtype found for qmu.results.
No classtype found for miscellaneous.dummy.
No classtype found for private.bamg.
No classtype found for private.bamg.mesh.
No classtype found for private.bamg.geometry.


In [ ]:
## Define inversion parameters
md.inversion.iscontrol = 1
md.inversion.maxsteps = 20
md.inversion.maxiter = 40
md.inversion.dxmin = 0.1
md.inversion.gttol = 1.0e-4

md.inversion.cost_functions = [101, 103, 501]
md.inversion.cost_functions_coefficients = np.ones((md.mesh.numberofvertices, 3))
md.inversion.cost_functions_coefficients[:,0] = 1
md.inversion.cost_functions_coefficients[:,1] = 1
md.inversion.cost_functions_coefficients[:,2] = 8e-15

md.inversion.control_parameters = ['FrictionCoefficient']
md.inversion.min_parameters = 1 * np.ones(md.mesh.numberofvertices, )
md.inversion.max_parameters = 200 * np.ones(md.mesh.numberofvertices, )

md.stressbalance.restol = 0.01
md.stressbalance.reltol = 0.1
md.stressbalance.abstol = np.nan

In [ ]:
## Solve
md.cluster.np = 2
# md.outputdefinition.definitions = []
# md.rifts.riftstruct = []
# md.rifts.riftproperties = []
# md.esa.transitions = []
# md.smb.requested_outputs
type(md.solidearth.settings)
# md = pyissm.model.execute.solve(md, 'Stressbalance')

In [ ]:
velocity = catalog.load_dataset('measures_insar_based_antarctica_ice_velocity_map', version = 'v2')
md.initialization.vx = pyissm.data.interp.xr_to_mesh(velocity, 'VX', md.mesh.x, md.mesh.y)

In [ ]:
valid_mask = np.isfinite(md.initialization.vx)

filled_all = pyissm.data.interp.points_to_mesh(
    data_x=md.mesh.x[valid_mask],
    data_y=md.mesh.y[valid_mask],
    data_values=md.initialization.vx[valid_mask],
    mesh_x=md.mesh.x,
    mesh_y=md.mesh.y,
    interpolation_type='nearest'
)

# Only replace NaNs
values_filled = md.initialization.vx.copy()
values_filled[~valid_mask] = filled_all[~valid_mask]

In [ ]:
pyissm.plot.plot_model_field(md, values_filled)